In [68]:
import numpy as np
import pandas as pd
import re
import string
import seaborn as sns
from wordcloud import STOPWORDS
from PIL import Image
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import nltk
import re
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_set =  set(stopwords.words('english'))
#we clean up unnecessary marks

nltk.download('punkt')  # 第一次使用时需要下载
from nltk.tokenize import word_tokenize
import string
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/laneylee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/laneylee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [40]:
result = pd.read_csv('result/all&suicide_with_prob/all_probs.csv')
result

,id,predictions,anger,annoyance,disappointment,disapproval,disgust,embarrassment,fear,grief,...,remorse,sadness,positive,neutral,class,text,token,sentiment,token_count,text_length
0,0,12.0,0.003140,0.013643,0.320729,0.105968,1.306766e-03,3.976417e-03,1.945175e-03,6.303866e-04,...,6.034370e-04,1.836595e-02,5.781246e-02,0.444644,almosthomeless,I literally don’t know where to ask this but I...,liter dont know ask serious question advic pls...,-0.063889,38.0,85.0
1,1,1.0,0.033857,0.866869,0.000738,0.083598,1.513196e-03,1.323251e-02,2.314404e-05,2.276647e-06,...,2.588070e-05,1.288332e-04,1.572072e-07,0.000009,almosthomeless,I'm almost 16 and am going to study med but I ...,im almost go studi med need take coach henc m...,0.136364,29.0,58.0
2,2,3.0,0.000549,0.299958,0.000882,0.639542,1.295966e-03,4.677313e-03,9.640865e-04,4.965249e-06,...,4.310008e-08,1.453962e-06,2.927398e-05,0.052072,almosthomeless,"I got a job, I go to orientation Tuesday. Tomo...",got job go orient tuesday tomorrow homeless we...,0.050451,83.0,171.0
3,3,1.0,0.002605,0.637379,0.008194,0.351810,3.153184e-08,6.816774e-07,1.871734e-08,3.432036e-09,...,1.738535e-09,2.225840e-08,6.092990e-07,0.000011,almosthomeless,Struggling over here to get everything lined u...,struggl get everyth line upim need refug seek ...,-0.002083,119.0,228.0
4,4,12.0,0.000372,0.001420,0.004618,0.008440,3.367238e-04,1.772046e-03,4.271182e-03,6.603977e-04,...,5.265729e-04,6.840330e-04,3.335223e-02,0.912119,almosthomeless,Ill be homeless officially Monday. Moving out ...,ill homeless offici monday move sunday hope st...,-0.100000,39.0,81.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14790,14047,3.0,0.000194,0.003100,0.272729,0.447826,5.954654e-03,1.449352e-03,4.251438e-02,3.976550e-03,...,4.438736e-03,8.515444e-02,3.094133e-03,0.071845,normal_positive,Luigi is the best character in super Mario 3D ...,luigi best charact super mario world disagre ...,0.666667,9.0,19.0
14791,14048,3.0,0.006142,0.040784,0.169939,0.467442,3.218773e-03,1.576891e-03,1.857177e-03,1.132213e-03,...,2.526511e-03,9.345067e-03,3.685362e-02,0.252023,normal_positive,"Man, rye bread with ham and cheese is really g...",man rye bread ham chees realli good tri sell r...,0.700000,12.0,22.0
14792,14049,1.0,0.234363,0.378303,0.111972,0.129934,6.946132e-02,4.589238e-02,1.125795e-04,7.796997e-04,...,4.606484e-03,2.616194e-03,1.306576e-02,0.008035,normal_positive,"Idk what to call this, but here’s a rant I thi...",idk call here rant think life go downhil never...,0.120000,30.0,53.0
14793,14050,1.0,0.003498,0.715970,0.043989,0.062357,6.876551e-02,7.745647e-02,1.038076e-04,1.041726e-04,...,3.678730e-03,2.400653e-02,1.788583e-05,0.000030,normal_positive,Important announcement! Axolotls are the best ...,import announc axolotl best creatur planetthat...,0.550000,9.0,18.0


In [41]:
def clean(text):
    text = str(text).lower()# 转换成小写
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[^a-zA-Z\s]', '', text)# filter non English char
    text = [word for word in text.split(' ') if word not in stopwords_set]
    text=" ".join(text)
#     text = [stemmer.stem(word) for word in text.split(' ')]
#     text=" ".join(text)
    return text

In [42]:
result['cleaned_text'] = result['text'].apply(clean)

In [43]:
result.groupby('class').agg(lambda x: x.nunique())['id']

class
adhd                997
almosthomeless      907
anxiety             992
assistance          912
bipolar             995
depression          993
eatingdisorders     991
get_motivated       687
normal_positive    2699
ocd                 993
ptsd                989
selfharm            996
stress              901
Name: id, dtype: int64

In [44]:
result.columns

Index(['id', 'predictions', 'anger', 'annoyance', 'disappointment',
       'disapproval', 'disgust', 'embarrassment', 'fear', 'grief',
       'nervousness', 'remorse', 'sadness', 'positive', 'neutral', 'class',
       'text', 'token', 'sentiment', 'token_count', 'text_length',
       'cleaned_text'],
      dtype='object')

In [45]:
emotion = ['anger', 'annoyance', 'disappointment',
       'disapproval', 'disgust', 'embarrassment', 'fear', 'grief',
       'nervousness', 'remorse', 'sadness', 'positive', 'neutral']

for i in emotion:
    print(i)
    print(result.groupby(['class']).describe()[i][['mean','std']])

anger
                     mean       std
class                              
adhd             0.026404  0.084364
almosthomeless   0.029683  0.091585
anxiety          0.016394  0.053660
assistance       0.021355  0.059774
bipolar          0.030582  0.096334
depression       0.041078  0.115673
eatingdisorders  0.018615  0.057943
get_motivated    0.023386  0.072877
normal_positive  0.040116  0.091335
ocd              0.024294  0.081746
ptsd             0.032092  0.095644
selfharm         0.052090  0.140105
stress           0.023789  0.073854
annoyance
                     mean       std
class                              
adhd             0.321259  0.351315
almosthomeless   0.225152  0.287771
anxiety          0.317384  0.354419
assistance       0.231737  0.298865
bipolar          0.269073  0.317428
depression       0.277799  0.333399
eatingdisorders  0.306724  0.334841
get_motivated    0.254336  0.321227
normal_positive  0.134668  0.185340
ocd              0.264810  0.318387
ptsd        

# 提取所有文本的keyword

In [46]:
# 注意力缺陷多动障碍 
keywords = {
    'adhd': ['adhd', 'vyvanse','medication', 'diagnosed'], 
    'almosthomeless': [ 'homeless', 'help', 'eviction', 'car'],
    'anxiety': ['anxiety', 'medication'],
    'assistance': ['assist', 'help',  'money'],
    'bipolar': ['bipolar',  'depression'],
    'depression': ['depression'],
    'eatingdisorders': [ 'foods', 'weight', 'disorder', 'eating lot', 'ate lot'],
    'get_motivated': ['energy', 'outcome','motivate'],
#     'normal_positive': ['good', 'best', 'happy', 'birthday', 'friend'],
    'ocd': [ 'ocd',  'obsessive'],
    'ptsd': [ 'therapist'],
    'selfharm': ['cut', 'self harm', 'blade','harm'],
    'stress': ['stress'],
    #'mask':['doubt', 'depress', 'trouble', 'pressure', 'sadness', 'grief'], # --> from paper, may not suit our purpose
    'suicide':['die','death','survive','suicide','kill']
}

In [47]:
keywords = {'mask':['doubt', 'depress', 'trouble', 'pressure', 'sadness', 'grief']}

In [48]:
mental_class = ['adhd', 'almosthomeless', 'anxiety', 'assistance', 'bipolar', 'depression', 'eatingdisorders', 'get_motivated', 'ocd', 'ptsd', 'normal_positive','selfharm', 'stress']
def count_keywords(text, keywords_dict):
    # 初始化一个字典来存储每个分类的关键词出现次数
    keyword_counts = {category:{} for category in keywords_dict}
    # 遍历每个分类和其关键词列表
    for category, words in keywords_dict.items():
        for word in words:
            # 计算每个关键词在文本中的出现次数，并累加到相应的分类下
            keyword_counts[category][word] = text.lower().count(word.lower())
    return keyword_counts

# 使用字典来存储每个类别的关键词出现次数
counts = {}
for category in mental_class:
    # 合并属于同一分类的所有文本
    category_text = ' '.join(result[result['class'] == category]['cleaned_text'])
    # 计算关键词出现次数
    counts[category] = count_keywords(category_text, keywords)



In [11]:
keywords.keys()

dict_keys(['mask'])

In [12]:
for j in mental_class:
    print(j)
    for i in counts:
        try:
            print(i,counts[i][j])
        except:
            pass

adhd
almosthomeless
anxiety
assistance
bipolar
depression
eatingdisorders
get_motivated
ocd
ptsd
normal_positive
selfharm
stress


In [13]:
# def remove_keyword(text,keywords,category):
#     try:
#         for i in keywords[category]:
#             text = text.lower()
#             text = text.replace(i,'[MASK]')
#     except:
#         text = str(text)
#     return text
# result['text_no_kw'] = '1'
# result['cleaned_text_no_kw'] = '1'
# for category in mental_class:
#     print(category)
#     result.loc[result['class'] == category, 'text_no_kw'] = result.loc[result['class'] == category, 'text'].apply(lambda x: remove_keyword(x, keywords, category))
#     result.loc[result['class']==category,'cleaned_text_no_kw'] = result.loc[result['class']==category,'cleaned_text'].apply(lambda x:remove_keyword(x,keywords,category))                                  

In [14]:
result.columns

Index(['id', 'predictions', 'anger', 'annoyance', 'disappointment',
       'disapproval', 'disgust', 'embarrassment', 'fear', 'grief',
       'nervousness', 'remorse', 'sadness', 'positive', 'neutral', 'class',
       'text', 'token', 'sentiment', 'token_count', 'text_length'],
      dtype='object')

In [15]:
# def remove_keyword_byall(text,keywords):
#     try:
#         for i in keywords:
#             if i == 'suicide':
#                 break
#             for j in keywords[i][:2]:
# #                 print(j)
#                 text = text.lower()
#                 text = text.replace(j,'[MASK]')
#     except:
#         text = str(text)
#     return text
# result['text_no_kw'] = result['text'].apply(lambda x: remove_keyword_byall(x, keywords))
# result['cleaned_text_no_kw'] = result['cleaned_text'].apply(lambda x:remove_keyword_byall(x,keywords))                               

In [16]:
# result[['id', 'anger', 'annoyance', 'disappointment',
#        'disapproval', 'disgust', 'embarrassment', 'fear', 'grief',
#        'nervousness', 'remorse', 'sadness', 'positive', 'neutral', 'class',
#      'sentiment', 
#        'text_no_kw', 'cleaned_text_no_kw']].to_csv('result/df_all_nokw.csv',index = False)

In [17]:
result.columns

Index(['id', 'predictions', 'anger', 'annoyance', 'disappointment',
       'disapproval', 'disgust', 'embarrassment', 'fear', 'grief',
       'nervousness', 'remorse', 'sadness', 'positive', 'neutral', 'class',
       'text', 'token', 'sentiment', 'token_count', 'text_length'],
      dtype='object')

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import pandas as pd
# 假设df_all是你的数据集DataFrame

df_all = result.copy()
df_all = df_all.dropna()
def train_multiclass_model(df_all):
    df_all = df_all.dropna()
    # 分割数据集为训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(
        df_all['cleaned_text'],  # 特征
        df_all['class'],  # 标签
        test_size=0.2,
        random_state=42
    )

    # 初始化TfidfVectorizer和LogisticRegression
    tfidf = TfidfVectorizer()
    # 多分类逻辑回归，使用多项式逻辑回归（'multinomial'）和l2正则化
    clf = LogisticRegression(multi_class='multinomial', solver='lbfgs')

    # 构建Pipeline
    model = Pipeline([('tfidf', tfidf), ('clf', clf)])

    # 训练模型
    model.fit(X_train, y_train)

    # 预测测试集
    y_pred = model.predict(X_test)

    # 计算和打印性能指标
    accuracy = accuracy_score(y_test, y_pred)
    print("模型准确率：", accuracy)

    # 由于是多分类问题，precision、recall和f1-score需要指定average参数
    # average='macro'计算所有类别的指标的未加权平均值
    precision = precision_score(y_test, y_pred, average='macro')
    print("精确率：", precision)

    recall = recall_score(y_test, y_pred, average='macro')
    print("召回率：", recall)

    f1 = f1_score(y_test, y_pred, average='macro')
    print("F1-score：", f1)

# 调用函数，传入DataFrame和子类别列表

train_multiclass_model(df_all)


模型准确率： 0.7515781656145563
精确率： 0.7607434975637773
召回率： 0.7060655529746229
F1-score： 0.7260592104817327


In [50]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


def train_multiclass_model_withemo(df_all):
    df_all = df_all.dropna()
    # 定义特征和标签
    features = [ 'anger', 'annoyance', 'disappointment',
       'disapproval', 'disgust', 'embarrassment', 'fear', 'grief',
       'nervousness', 'remorse', 'sadness', 'positive', 'neutral',
         'cleaned_text']  # 添加你的数值特征列名
    label = 'class'

    # 分割数据集为训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(
        df_all[features],  # 特征
        df_all[label],  # 标签
        test_size=0.2,
        random_state=42
    )

    # 为文本特征和数值特征定义不同的处理器
    text_features = 'cleaned_text'  # 文本列名
    numerical_features = [ 'anger', 'annoyance', 'disappointment',
       'disapproval', 'disgust', 'embarrassment', 'fear', 'grief',
       'nervousness', 'remorse', 'sadness', 'positive', 'neutral']  # 数值列名列表

    preprocessor = ColumnTransformer(
        transformers=[
            ('text', TfidfVectorizer(), text_features),
            ('num', StandardScaler(), numerical_features)
        ]
    )

    # 多分类逻辑回归，使用多项式逻辑回归（'multinomial'）和l2正则化
    clf = LogisticRegression(multi_class='multinomial', solver='lbfgs')

    # 构建Pipeline
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', clf)
    ])

    # 在训练集上训练模型
    model.fit(X_train, y_train)

    # 在测试集上进行预测
    y_pred = model.predict(X_test)
        # 计算和打印性能指标
    accuracy = accuracy_score(y_test, y_pred)
    print("模型准确率：", accuracy)

    # 由于是多分类问题，precision、recall和f1-score需要指定average参数
    # average='macro'计算所有类别的指标的未加权平均值
    precision = precision_score(y_test, y_pred, average='macro')
    print("精确率：", precision)

    recall = recall_score(y_test, y_pred, average='macro')
    print("召回率：", recall)

    f1 = f1_score(y_test, y_pred, average='macro')
    print("F1-score：", f1)
    return X_train, X_test, y_train, y_test # need the train_test for validation!
    
X_train, X_test, y_train, y_test  = train_multiclass_model_withemo(result)# with emotional

模型准确率： 0.7482361678425548
精确率： 0.7486317802940796
召回率： 0.704647157168852
F1-score： 0.7217802376052469


In [ ]:
X_train

In [126]:
result.columns

Index(['id', 'predictions', 'anger', 'annoyance', 'disappointment',
       'disapproval', 'disgust', 'embarrassment', 'fear', 'grief',
       'nervousness', 'remorse', 'sadness', 'positive', 'neutral', 'class',
       'text', 'token', 'sentiment', 'token_count', 'text_length',
       'cleaned_text', 'text_no_kw', 'cleaned_text_no_kw'],
      dtype='object')

original
模型准确率： 0.7515781656145563
精确率： 0.7607434975637773
召回率： 0.7060655529746229
F1-score： 0.7260592104817327

with score
模型准确率： 0.7482361678425548
精确率： 0.7486317802940796
召回率： 0.704647157168852
F1-score： 0.7217802376052469

# 替换关键词

In [406]:
# from nltk.corpus import wordnet as wn

# def get_synonyms(word):
#     synonyms = set()
#     for syn in wn.synsets(word):
#         for lemma in syn.lemmas():
#             synonyms.add(lemma.name())
#     return list(synonyms)


# # 生成包含近义词的新字典
# keywords_synonyms = {category: {word: get_synonyms(word) for word in words} for category, words in keywords.items()}

# # 示例输出
# for category, words in keywords_synonyms.items():
#     print(f"{category}:")
#     for word, synonyms in words.items():
#         print(f"  {word}: {synonyms}")
 # doesnt work well!

In [392]:
# replace_kw = keywords_synonyms 

In [51]:
replace_kw = {
  'adhd': {
    'adhd': ['Attention Deficit Hyperactivity Disorder'],
    'vyvanse': ['lisdexamfetamine', 'stimulant medication'],
    'medication': ['medicine', 'drug', 'treatment'],
    'diagnosed': ['identified', 'detected', 'recognized']
  },
  'almosthomeless': {
    'homeless': ['unhoused', 'homelessness', 'displaced'],
    'help': ['assist', 'aid', 'support'],
    'eviction': ['displacement', 'expulsion', 'removal'],
    'car': ['automobile', 'vehicle', 'motorcar']
  },
  'anxiety': {
    'anxiety': ['nervousness', 'worry', 'apprehension'],
    'medication': ['medicine', 'drug', 'treatment']
  },
  'assistance': {
    'assist': ['help', 'aid', 'support'],
    'help': ['assist', 'aid', 'support'],
    'money': ['cash', 'funds', 'capital']
  },
  'bipolar': {
    'bipolar': ['manic-depressive', 'bipolar disorder'],
    'depression': ['sadness', 'melancholy', 'despondency']
  },
  'depression': {
    'depression': ['sadness', 'melancholy', 'despondency']
  },
  'eatingdisorders': {
    'foods': ['meals', 'edibles', 'nourishment'],
    'weight': ['mass', 'heaviness', 'bulk'],
    'disorder': ['condition', 'disease', 'ailment'],
    'eating lot': ['overeating', 'binge eating'],
    'ate lot': ['overate', 'binged']
  },
  'get_motivated': {
    'energy': ['vitality', 'vigor', 'dynamism'],
    'outcome': ['result', 'consequence', 'effect'],
    'motivate': ['inspire', 'encourage', 'stimulate']
  },
  'ocd': {
    'ocd': ['Obsessive-Compulsive Disorder', 'compulsion'],
    'obsessive': ['compulsive', 'fixated', 'obsessed']
  },
  'ptsd': {
    'therapist': ['counselor', 'psychotherapist', 'psychologist']
  },
  'selfharm': {
    'cut': ['slice', 'slash', 'gash'],
    'self harm': ['self-injury', 'self-mutilation'],
    'blade': ['knife', 'razor', 'cutter'],
    'harm': ['injury', 'damage', 'hurt']
  },
  'stress': {
    'stress': ['pressure', 'tension', 'strain']
  },
  'suicide': {
    'die': ['pass away', 'perish', 'expire'],
    'death': ['demise', 'fatality', 'mortality'],
    'survive': ['live through', 'outlast', 'endure'],
    'suicide': ['self-destruction', 'self-killing'],
    'kill': ['murder', 'slay', 'assassinate']
  }
}
# work with 1-gram!

In [52]:
# replace_kw = {
#     'adhd': {
#         'adhd': ['Attention Deficit Hyperactivity Disorder'],
#         'medication': ['medicine', 'treatment', 'drug'],
#         'diagnosed': ['identified', 'detected'],
#         'vyvanse': ['stimulant medication', 'ADHD drug'],
#         'work': ['labor', 'task', 'job'],
#         'time': ['period', 'duration'],
#         'life': ['existence', 'living'],
#         'things': ['items', 'objects'],
#         'body': ['physique', 'anatomy']
#     },
#     'almosthomeless': {
#         'get job': ['find employment', 'secure a position'],
#         'homeless': ['without a home', 'unhoused'],
#         'kicked family': ['expelled by relatives', 'disowned'],
#         'uber': ['ride-sharing service', 'taxi alternative'],
#         'eviction': ['expulsion', 'dislodgement'],
#         'rate': ['speed', 'pace'],
#         'position': ['job', 'role'],
#         'shit': ['crap', 'nonsense'],
#         'snap': ['break', 'crack'],
#         'car': ['automobile', 'vehicle']
#     },
#     'anxiety': {
#         'anxiety': ['nervousness', 'worry'],
#         'medication': ['medicine', 'treatment'],
#         'fucking anxiety': ['intense worry', 'severe nervousness'],
#         'insane': ['crazy', 'mad'],
#         'health anxiety': ['hypochondria', 'medical worry'],
#         'm anxiety': ['my worry', 'personal nervousness'],
#         'day': ['24 hours', 'daily'],
#         'bad': ['poor', 'negative'],
#         'worst': ['most terrible', 'least good']
#     },
#     'assistance': {
#     'able get': ['capable of obtaining', 'can secure'],
#     'get help': ['receive assistance', 'obtain support'],
#     'little help': ['minor assistance', 'slight aid'],
#     'money': ['funds', 'currency'],
#     'accepted': ['approved', 'agreed'],
# },
# 'bipolar': {
#     'bipolar': ['manic-depressive illness', 'bipolar disorder'],
#     'hunger': ['appetite', 'craving'],
#     'm meds': ['my medication', 'prescribed drugs'],
#     'get sleep': ['obtain rest', 'sleep well'],
#     'going depression': ['entering into depression', 'becoming depressed'],
#     'despair': ['hopelessness', 'despondency'],
#     'normal time': ['usual period', 'regular time']
# },
# 'depression': {
#     'love gone': ['lost love', 'end of romance'],
#     'felt need': ['experienced necessity', 'felt requirement'],
#     'feeling despair': ['feeling hopeless', 'sense of despair'],
#     'afraid life': ['fearful of living', 'scared of life'],
#     'wanted friends': ['desired companions', 'sought friendship'],
#     'stress': ['stressful', 'tense'],
# },
# 'eatingdisorders': {
#     'eating': ['consuming food', 'ingestion'],
#     'foods': ['meals', 'edibles'],
#     'weight': ['mass', 'body mass'],
#     'times': ['occasions', 'instances'],
#     'years': ['annum', 'calendar years'],
#     'comfortable': ['at ease', 'content'],
#     'ate': ['consumed', 'ingested'],
#     'eats': ['consumes', 'ingests'],
#     'disorder': ['condition', 'ailment'],
#     'eating lot': ['overeating', 'binge eating'],
#     'ate lot': ['overate', 'consumed much'],
#     'recovery': ['healing', 'recuperation'],
#     'eating meals': ['consuming food', 'meal intake']
# },
# 'get_motivated': {
#     'energy': ['vitality', 'power'],
#     'outcome': ['result', 'consequence'],
#     'normal day': ['ordinary day', 'usual day'],
#     'working day': ['day of work', 'business day'],
#     'get job': ['obtain employment', 'secure a position'],
#     'day cost': ['expense of the day', 'daily expenditure'],
#     'reevaluate': ['reassess', 'review']
# }
# }

In [53]:
import random

# 假设df是你的DataFrame，'class'是其中的一个列名，'text'是包含文本的列名
def replace_sys(word,text,synonyms,row,new_rows):
    try:
        if word in text.lower():
            # 随机决定是否替换以及使用哪个近义词
            if random.random() < 0.7:  # 有50%的概率进行替换
                synonym = random.choice(synonyms)  # 随机选择一个近义词
                # 替换文本中的'adhd'，这里使用正则表达式以匹配单词边界，避免替换子字符串
                new_text = re.sub(r'\badhd\b', synonym, text, flags=re.IGNORECASE)
                # 创建一个新行，这里使用.copy()来确保我们不会意外修改原始DataFrame
                new_row = row.copy()
                new_row['cleaned_text'] = new_text  # 更新文本
                new_rows.append(new_row)  # 添加到列表
    except:
        pass
# 近义词列表
def augment_df(result,category):
    synonyms_dict = replace_kw[category]

    # 筛选出'class'列为'adhd'的行
    category_rows = result[result['class'] == category]

    # 用于保存修改后的行的列表
    new_rows = []

    for index, row in category_rows.iterrows():
        text = row['cleaned_text']  # 假设文本列名为'text'
        for word in synonyms_dict.keys():
            synonyms = synonyms_dict[word]
            print(word,synonyms)
            replace_sys(word,text,synonyms,row,new_rows)

    # 将新行列表转换为DataFrame
    new_rows_df = pd.DataFrame(new_rows)
    new_rows_df['augmented'] = 1
    # # 如果需要，重置索引
    new_rows_df.reset_index(drop=True, inplace=True)

    # # 将新行添加到原始DataFrame中
    # df_augmented = pd.concat([df, new_rows_df], ignore_index=True)

    # 显示结果
    # print(new_rows_df)
    return new_rows_df


In [54]:
Train = pd.concat([X_train,y_train],axis = 1)
Train = Train.reset_index(drop=True)
Train['augmented'] = 0 # 标记是否为增强
Train

,anger,annoyance,disappointment,disapproval,disgust,embarrassment,fear,grief,nervousness,remorse,sadness,positive,neutral,cleaned_text,class,augmented
0,0.000377,0.016068,0.035070,0.202846,7.202113e-04,0.000912,4.797461e-03,3.573275e-04,1.274492e-02,2.495390e-04,2.017067e-03,4.202968e-01,3.035441e-01,great news father recovering corona hospital ...,normal_positive,0
1,0.008999,0.056374,0.166283,0.652791,4.573573e-04,0.001685,1.878334e-05,2.809609e-04,4.235959e-04,6.475875e-04,8.465126e-03,9.562749e-02,7.947659e-03,good morning people put cereal first oh also g...,normal_positive,0
2,0.014021,0.190706,0.130012,0.106261,4.239085e-04,0.554364,4.157982e-04,1.068727e-05,2.324627e-03,2.911549e-07,3.717595e-04,4.290392e-06,1.084685e-03,first time sub ive lexapro days took panic an...,anxiety,0
3,0.007742,0.256413,0.003211,0.731839,1.681962e-07,0.000792,3.692591e-07,2.726226e-10,2.304939e-11,2.929336e-10,3.892165e-09,5.178214e-11,2.781450e-06,throwaway account im embarrassed talk thishi i...,adhd,0
4,0.000031,0.026559,0.092793,0.238683,2.910692e-05,0.165362,3.603083e-02,1.113617e-04,3.088459e-01,1.150014e-05,1.333104e-02,1.139902e-04,1.180985e-01,hey see lot posts afraid share diagnosis other...,bipolar,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10763,0.000294,0.001740,0.257945,0.173793,1.649276e-04,0.000888,3.172238e-04,7.567086e-04,5.395566e-01,8.791842e-05,2.104666e-02,1.363963e-04,3.273481e-03,lost mum october sunday gone got call say part...,stress,0
10764,0.008455,0.093432,0.013903,0.061975,8.909390e-03,0.004420,2.223132e-03,1.478796e-03,1.503773e-03,5.477535e-03,1.523704e-02,6.347883e-01,1.481973e-01,favourite forbidden snack kid mine wall also l...,normal_positive,0
10765,0.248555,0.542076,0.009714,0.197001,1.931459e-05,0.000025,1.810165e-04,6.183177e-06,8.342523e-07,1.268263e-08,1.591701e-04,3.819113e-07,2.262851e-03,something extremely stupidabout something alre...,stress,0
10766,0.000345,0.998720,0.000035,0.000524,8.308086e-09,0.000376,1.297842e-07,5.130466e-09,7.288504e-11,1.147119e-11,4.563700e-11,2.602309e-10,9.876947e-08,take wellbutrin social anxiety adhd gad thoug...,anxiety,0


In [55]:
df_augmented = Train.copy()

for i in mental_class:
    if i == 'normal_positive':
        continue
    new_rows_df = augment_df(df_augmented,i)
    df_augmented = pd.concat([df_augmented, new_rows_df], ignore_index=True)

adhd ['Attention Deficit Hyperactivity Disorder']
vyvanse ['lisdexamfetamine', 'stimulant medication']
medication ['medicine', 'drug', 'treatment']
diagnosed ['identified', 'detected', 'recognized']
adhd ['Attention Deficit Hyperactivity Disorder']
vyvanse ['lisdexamfetamine', 'stimulant medication']
medication ['medicine', 'drug', 'treatment']
diagnosed ['identified', 'detected', 'recognized']
adhd ['Attention Deficit Hyperactivity Disorder']
vyvanse ['lisdexamfetamine', 'stimulant medication']
medication ['medicine', 'drug', 'treatment']
diagnosed ['identified', 'detected', 'recognized']
adhd ['Attention Deficit Hyperactivity Disorder']
vyvanse ['lisdexamfetamine', 'stimulant medication']
medication ['medicine', 'drug', 'treatment']
diagnosed ['identified', 'detected', 'recognized']
adhd ['Attention Deficit Hyperactivity Disorder']
vyvanse ['lisdexamfetamine', 'stimulant medication']
medication ['medicine', 'drug', 'treatment']
diagnosed ['identified', 'detected', 'recognized']
adhd 

energy ['vitality', 'vigor', 'dynamism']
outcome ['result', 'consequence', 'effect']
motivate ['inspire', 'encourage', 'stimulate']
energy ['vitality', 'vigor', 'dynamism']
outcome ['result', 'consequence', 'effect']
motivate ['inspire', 'encourage', 'stimulate']
energy ['vitality', 'vigor', 'dynamism']
outcome ['result', 'consequence', 'effect']
motivate ['inspire', 'encourage', 'stimulate']
energy ['vitality', 'vigor', 'dynamism']
outcome ['result', 'consequence', 'effect']
motivate ['inspire', 'encourage', 'stimulate']
energy ['vitality', 'vigor', 'dynamism']
outcome ['result', 'consequence', 'effect']
motivate ['inspire', 'encourage', 'stimulate']
energy ['vitality', 'vigor', 'dynamism']
outcome ['result', 'consequence', 'effect']
motivate ['inspire', 'encourage', 'stimulate']
energy ['vitality', 'vigor', 'dynamism']
outcome ['result', 'consequence', 'effect']
motivate ['inspire', 'encourage', 'stimulate']
energy ['vitality', 'vigor', 'dynamism']
outcome ['result', 'consequence', '

In [56]:
df_augmented = pd.concat([Train, df_augmented], ignore_index=True)

In [57]:
df_augmented.columns

Index(['anger', 'annoyance', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'fear', 'grief', 'nervousness', 'remorse', 'sadness',
       'positive', 'neutral', 'cleaned_text', 'class', 'augmented'],
      dtype='object')

In [58]:
df_augmented = df_augmented.drop_duplicates().dropna().reset_index(drop=True)

In [59]:
df_augmented

,anger,annoyance,disappointment,disapproval,disgust,embarrassment,fear,grief,nervousness,remorse,sadness,positive,neutral,cleaned_text,class,augmented
0,0.000377,0.016068,0.035070,0.202846,7.202113e-04,0.000912,4.797461e-03,3.573275e-04,1.274492e-02,2.495390e-04,2.017067e-03,4.202968e-01,0.303544,great news father recovering corona hospital ...,normal_positive,0
1,0.008999,0.056374,0.166283,0.652791,4.573573e-04,0.001685,1.878334e-05,2.809609e-04,4.235959e-04,6.475875e-04,8.465126e-03,9.562749e-02,0.007948,good morning people put cereal first oh also g...,normal_positive,0
2,0.014021,0.190706,0.130012,0.106261,4.239085e-04,0.554364,4.157982e-04,1.068727e-05,2.324627e-03,2.911549e-07,3.717595e-04,4.290392e-06,0.001085,first time sub ive lexapro days took panic an...,anxiety,0
3,0.007742,0.256413,0.003211,0.731839,1.681962e-07,0.000792,3.692591e-07,2.726226e-10,2.304939e-11,2.929336e-10,3.892165e-09,5.178214e-11,0.000003,throwaway account im embarrassed talk thishi i...,adhd,0
4,0.000031,0.026559,0.092793,0.238683,2.910692e-05,0.165362,3.603083e-02,1.113617e-04,3.088459e-01,1.150014e-05,1.333104e-02,1.139902e-04,0.118098,hey see lot posts afraid share diagnosis other...,bipolar,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14367,0.000622,0.230216,0.666321,0.008106,1.344669e-06,0.000055,8.516072e-02,9.733274e-07,9.246373e-03,4.969603e-10,2.199606e-05,4.117075e-08,0.000248,hi guysim amp last months im feeling tired ti...,stress,1
14368,0.000959,0.045486,0.254379,0.493968,3.246395e-04,0.001441,4.279843e-03,8.907627e-05,2.159943e-02,7.146539e-07,4.351602e-02,2.794524e-04,0.133679,im financially stable dont really study hard d...,stress,1
14369,0.308575,0.678246,0.000017,0.012048,3.208358e-05,0.001019,8.397480e-07,5.879250e-05,1.408063e-08,3.109309e-07,3.490410e-07,1.029593e-08,0.000003,want try get grip stress ive stressed life nev...,stress,1
14370,0.001555,0.009520,0.037964,0.021233,2.047394e-03,0.015122,4.014706e-05,1.170717e-05,2.921906e-03,1.050733e-06,1.778206e-04,7.150104e-02,0.837906,hello long time see another rstress posti want...,stress,1


In [60]:
df_augmented.groupby('class').describe()['augmented']

,count,mean,std,min,25%,50%,75%,max
class,,,,,,,,
adhd,1330.0,0.416541,0.493171,0.0,0.0,0.0,1.0,1.0
almosthomeless,1066.0,0.388368,0.487608,0.0,0.0,0.0,1.0,1.0
anxiety,1164.0,0.340206,0.473982,0.0,0.0,0.0,1.0,1.0
assistance,1111.0,0.360036,0.480227,0.0,0.0,0.0,1.0,1.0
bipolar,1031.0,0.243453,0.429374,0.0,0.0,0.0,0.0,1.0
depression,923.0,0.158180,0.365107,0.0,0.0,0.0,0.0,1.0
eatingdisorders,1143.0,0.320210,0.466761,0.0,0.0,0.0,1.0,1.0
get_motivated,428.0,0.177570,0.382598,0.0,0.0,0.0,0.0,1.0
normal_positive,2161.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [61]:
y_test.value_counts()

class
normal_positive    516
ocd                209
adhd               207
depression         202
eatingdisorders    196
anxiety            195
ptsd               195
selfharm           194
bipolar            190
assistance         174
almosthomeless     173
stress             151
get_motivated       91
Name: count, dtype: int64

In [62]:
from sklearn.metrics import classification_report


# 分割数据集为训练集和测试集
def add_noise(df,column):
    mu = 0
    sigma = 1

    # 生成与列同长度的高斯噪音
    noise = np.random.normal(mu, sigma, size=len(df))
    df[column]+= noise
    # 给列添加噪音

    
def LR_with_emo_feature(df_augmented,y_test,X_test,if_numerical = True):
    text_features = 'cleaned_text'  # 文本列名
    numerical_features = [ 'anger', 'annoyance', 'disappointment',
       'disapproval', 'disgust', 'embarrassment', 'fear', 'grief',
       'nervousness', 'remorse', 'sadness', 'positive', 'neutral']  # 数值列名列表



    X_train, y_train = df_augmented[['anger', 'annoyance', 'disappointment',
       'disapproval', 'disgust', 'embarrassment', 'fear', 'grief',
       'nervousness', 'remorse', 'sadness', 'positive', 'neutral','cleaned_text']],df_augmented['class']
    if if_numerical:
        preprocessor = ColumnTransformer(
            transformers=[
                ('text', TfidfVectorizer(), text_features),
                ('num', StandardScaler(), numerical_features)
            ]
        )
    else:
        preprocessor = ColumnTransformer(
            transformers=[
                ('text', TfidfVectorizer(), text_features)
            ]
        )

    # 多分类逻辑回归，使用多项式逻辑回归（'multinomial'）和l2正则化
    clf = LogisticRegression(multi_class='multinomial', solver='lbfgs')

    # 构建Pipeline
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', clf)
    ])

    # 在训练集上训练模型
    model.fit(X_train, y_train)

    # 在测试集上进行预测
    y_pred = model.predict(X_test)
        # 计算和打印性能指标
    accuracy = accuracy_score(y_test, y_pred)
    print("模型准确率：", accuracy)

    # 由于是多分类问题，precision、recall和f1-score需要指定average参数
    # average='macro'计算所有类别的指标的未加权平均值
    precision = precision_score(y_test, y_pred, average='macro')
    print("精确率：", precision)

    recall = recall_score(y_test, y_pred, average='macro')
    print("召回率：", recall)

    f1 = f1_score(y_test, y_pred, average='macro')
    print("F1-score：", f1)

    report = classification_report(y_test, y_pred, target_names=df_augmented['class'].unique())
    print(report)
    return report

In [63]:
# df_augmented_with_noise = df_augmented.copy()

# numerical_features = [ 'anger', 'annoyance', 'disappointment',
#        'disapproval', 'disgust', 'embarrassment', 'fear', 'grief',
#        'nervousness', 'remorse', 'sadness', 'positive', 'neutral'] 
# for i in numerical_features :
#     add_noise(df_augmented_with_noise[(df_augmented['augmented']==1)],i)
# df_augmented_with_noise

In [64]:
result_with_score = LR_with_emo_feature(df_augmented,y_test,X_test,if_numerical = True)

模型准确率： 0.7571481619012254
精确率： 0.7511894939234331
召回率： 0.7144635232816772
F1-score： 0.7281209131361603
                 precision    recall  f1-score   support

normal_positive       0.70      0.78      0.74       207
        anxiety       0.72      0.74      0.73       173
           adhd       0.69      0.67      0.68       195
        bipolar       0.75      0.78      0.77       174
     assistance       0.81      0.63      0.71       190
            ocd       0.60      0.60      0.60       202
       selfharm       0.91      0.88      0.90       196
  get_motivated       0.64      0.47      0.54        91
           ptsd       0.77      0.99      0.87       516
eatingdisorders       0.84      0.69      0.76       209
         stress       0.75      0.62      0.67       195
     depression       0.82      0.83      0.83       194
 almosthomeless       0.76      0.62      0.68       151

       accuracy                           0.76      2693
      macro avg       0.75      0.71    

In [403]:
# result_with_noisyscore = LR_with_emo_feature(df_augmented_with_noise,y_test,X_test,if_numerical = True)

In [65]:
LR_with_emo_feature(df_augmented,y_test,X_test,if_numerical = False)

模型准确率： 0.7564054957296695
精确率： 0.7566297699999053
召回率： 0.7124570009016418
F1-score： 0.7282521446391801
                 precision    recall  f1-score   support

normal_positive       0.70      0.77      0.74       207
        anxiety       0.72      0.75      0.73       173
           adhd       0.69      0.65      0.67       195
        bipolar       0.74      0.78      0.76       174
     assistance       0.80      0.63      0.71       190
            ocd       0.61      0.61      0.61       202
       selfharm       0.91      0.88      0.89       196
  get_motivated       0.70      0.43      0.53        91
           ptsd       0.76      0.98      0.85       516
eatingdisorders       0.86      0.69      0.77       209
         stress       0.76      0.65      0.70       195
     depression       0.82      0.82      0.82       194
 almosthomeless       0.78      0.62      0.69       151

       accuracy                           0.76      2693
      macro avg       0.76      0.71    

'                 precision    recall  f1-score   support\n\nnormal_positive       0.70      0.77      0.74       207\n        anxiety       0.72      0.75      0.73       173\n           adhd       0.69      0.65      0.67       195\n        bipolar       0.74      0.78      0.76       174\n     assistance       0.80      0.63      0.71       190\n            ocd       0.61      0.61      0.61       202\n       selfharm       0.91      0.88      0.89       196\n  get_motivated       0.70      0.43      0.53        91\n           ptsd       0.76      0.98      0.85       516\neatingdisorders       0.86      0.69      0.77       209\n         stress       0.76      0.65      0.70       195\n     depression       0.82      0.82      0.82       194\n almosthomeless       0.78      0.62      0.69       151\n\n       accuracy                           0.76      2693\n      macro avg       0.76      0.71      0.73      2693\n   weighted avg       0.76      0.76      0.75      2693\n'

In [66]:
df_augmented.to_csv('result/df_all_augmented.csv',index = False)

In [67]:
pd.concat([X_test,y_test],axis = 1).to_csv('result/df_test_augmented.csv',index = False)

In [414]:
pd.concat([X_test,y_test],axis = 1).columns

Index(['anger', 'annoyance', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'fear', 'grief', 'nervousness', 'remorse', 'sadness',
       'positive', 'neutral', 'cleaned_text', 'class'],
      dtype='object')

In [415]:
df_augmented.columns

Index(['anger', 'annoyance', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'fear', 'grief', 'nervousness', 'remorse', 'sadness',
       'positive', 'neutral', 'cleaned_text', 'class', 'augmented'],
      dtype='object')